In [2]:
# Import necessary libraries
from numpy import array
import pandas as pd
import numpy as np
from datetime import datetime
from numpy import asarray
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit, KFold
import random
from sklearn.utils import shuffle
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
import tensorflow as tf
import pickle
import csv
from dateutil.relativedelta import relativedelta

In [39]:
# Read CPI data from CSV file
gdp_history_data = pd.read_csv('../input_data/gdp_yearly_1949_2022.csv')
gdp_data = gdp_history_data.copy()
# Setting idex as Dates column
gdp_data.set_index('Year', inplace=True)
gdp_history_data.tail()

,Year,Change in %
69,2018,1.4
70,2019,1.6
71,2020,-10.4
72,2021,8.7
73,2022,4.3


In [29]:
# Scale data between -1 and 1
gdp_scaler = MinMaxScaler()
scaled_gdp = asarray(gdp_data['Change in %']).reshape(-1, 1)
scaled_gdp = gdp_scaler.fit_transform(scaled_gdp)
# Omit the last 50 observations for out-of-sample forecast
gdp_sample_forecast_input = scaled_gdp[len(scaled_gdp) - 10:, 0]
scaled_gdp = scaled_gdp[:, 0]

# let's transform our remaning data into a univariate supervised learning problem
# Functions transforms our time series sequence into a supervised leaning problem
# Transform data into a univariate supervised learning problem
def split_sequences(sequence, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequence) - (n_steps_in + n_steps_out)):
        append_X = []
        append_y = []
        for j in range(n_steps_in):
            append_X.append(sequence[i + j])
        for k in range(n_steps_out):
            append_y.append(sequence[i + n_steps_in + k + 1])

        X.append(append_X)
        y.append(append_y)

    return np.array(X), np.array(y)

In [30]:
# Set the number of lags and forecast periods
n_steps_in = 10
n_steps_out = 5

# Create sequences for input (X) and output (y)
X, y = split_sequences(scaled_gdp, n_steps_in, n_steps_out)

# Split data into training and testing 80:20
total_rows = X.shape[0]
train_rows = int(total_rows * 0.8)
test_rows = total_rows - train_rows

# Obtain training and testing data
X_train = X[:train_rows]
X_test = X[train_rows:]
y_train = y[:train_rows]
y_test = y[train_rows:]

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
y_train = y_train.reshape(y_train.shape[0], y_train.shape[1], 1)

In [35]:
# Build and train the LSTM model

# Initialize a sequential model
model = Sequential()

# Add the first LSTM layer with 75 units, ReLU activation, and input shape (n_steps_in, 1)
# Return sequences as there is another LSTM layer following
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps_in, 1)))

# Add the second LSTM layer with 75 units and ReLU activation
# Does not return sequences as it is the final LSTM layer
model.add(LSTM(50, activation='relu', return_sequences=False))

# Add a dropout layer with a dropout rate of 0.2 to prevent overfitting
model.add(Dropout(0.5))

# Add a dense (fully connected) layer with n_steps_out units for prediction
model.add(Dense(n_steps_out))

# Compile the model with mean squared error loss and the Adam optimizer
model.compile(loss='mse', optimizer='adam')

training_model = model.fit(X_train, y_train, epochs=50, verbose=1)

Epoch 1/50
2/2 [==============================] - 4s 43ms/step - loss: 0.4443
Epoch 2/50
2/2 [==============================] - 0s 29ms/step - loss: 0.4276
Epoch 3/50
2/2 [==============================] - 0s 24ms/step - loss: 0.4062
Epoch 4/50
2/2 [==============================] - 0s 20ms/step - loss: 0.3861
Epoch 5/50
2/2 [==============================] - 0s 17ms/step - loss: 0.3645
Epoch 6/50
2/2 [==============================] - 0s 27ms/step - loss: 0.3439
Epoch 7/50
2/2 [==============================] - 0s 19ms/step - loss: 0.3197
Epoch 8/50
2/2 [==============================] - 0s 41ms/step - loss: 0.2731
Epoch 9/50
2/2 [==============================] - 0s 26ms/step - loss: 0.2399
Epoch 10/50
2/2 [==============================] - 0s 30ms/step - loss: 0.1955
Epoch 11/50
2/2 [==============================] - 0s 16ms/step - loss: 0.1476
Epoch 12/50
2/2 [==============================] - 0s 15ms/step - loss: 0.1268
Epoch 13/50
2/2 [==============================] - 0s 16ms/st

In [38]:
# Perform out-of-sample forecast
gdp_sample_forecast_input = asarray(gdp_sample_forecast_input).reshape(1, n_steps_in)
dpg_sample_forecast = model.predict(gdp_sample_forecast_input, verbose=0)

# Inversely scale the forecasted data and save it to a CSV file
gdp_list_forecast = gdp_scaler.inverse_transform(dpg_sample_forecast).tolist()
print('Predicted GDP Growth for next 10 Years: ', gdp_list_forecast[0])

Predicted GDP Growth for next 10 Years:  [0.3031187653541565, 0.7717838883399963, 0.20025081932544708, 0.08234395831823349, 0.4562389850616455]


In [44]:
gdp_list_forecast

[[0.3031187653541565,
  0.7717838883399963,
  0.20025081932544708,
  0.08234395831823349,
  0.4562389850616455]]

In [46]:
# Generate a list of the next 5 years
next_5_years = [2023 + i for i in range(1, 6)]
predicted_gdp_growth = pd.DataFrame({'Year':next_5_years, 'Change in %':gdp_list_forecast[0]})
predicted_gdp_growth['Change in %'] = predicted_gdp_growth['Change in %'].round(2)
predicted_gdp_growth.to_csv('../output_data/GDP_Growth_Over_next_5_Years.csv', index=False)
predicted_gdp_growth

,Year,Change in %
0,2024,0.30
1,2025,0.77
2,2026,0.20
3,2027,0.08
4,2028,0.46


In [47]:
gdp_history_data['Change in %'] = gdp_history_data['Change in %'].round(2)
predicted_gdp_growth.to_csv('../output_data/GDP_Growth_Over_next_5_Years.csv', index=False)
gdp_history_data.head()

,Year,Change in %
0,1949,3.3
1,1950,3.3
2,1951,3.7
3,1952,1.5
4,1953,5.6
